<a href="https://colab.research.google.com/github/marcelofschiavo/client_segmentation/blob/main/client_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Segmentação de Clientes (Clustering Não-Supervisionado)

Objetivo: Uma empresa como a @empresa não pode tratar todos os seus milhões de clientes da mesma forma. Este projeto busca usar Machine Learning Não-Supervisionado (Clustering) para identificar perfis (personas) de clientes com base em seu comportamento de compra.

Problema de Negócio: "Quais são os principais perfis de clientes que compram nossos produtos? Como podemos criar ações de marketing e retenção personalizadas para cada grupo?"

Metodologia:

ETL Simulado: Criação de um dataset fictício (mas realista) de comportamento de clientes.

Pré-Processamento (StandardScaler): Etapa crucial do clustering para padronizar a escala das variáveis.

Descoberta (Método do Cotovelo): Encontrar o número "ideal" de clusters (K).

Modelagem (K-Means): Treinar o modelo de clustering para agrupar os clientes.

Análise de Personas (Psicologia): Interpretar as médias de cada cluster para criar "personas" de negócio.

Bibliotecas: pandas (manipulação), sklearn (para StandardScaler, KMeans, PCA) e seaborn (visualização).

In [ ]:
# Importar bibliotecas fundamentais
import pandas as pd
import numpy as np

# Bibliotecas de Visualização (EDA)
import seaborn as sns
import matplotlib.pyplot as plt

# Bibliotecas de ML (Clustering)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA # Para visualização 2D

# Configuração
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
# Usamos seed(42) para garantir que nossos dados aleatórios sejam sempre os mesmos
np.random.seed(42)

print("Bibliotecas carregadas com sucesso.")

Etapa 1: Criação dos Dados (ETL Simulado)

Vamos criar um dataset fictício de 300 clientes. Para segmentar clientes, usamos um modelo comportamental famoso chamado RFM (Recência, Frequência, Valor Monetário), mas com um toque extra de psicologia:

dias_desde_ultima_compra: (Recência) Cliente ativo ou "dormindo"?

gasto_total_ano: (Monetário) Cliente gasta muito ou pouco?

frequencia_compras_ano: (Frequência) Cliente compra sempre ou raramente?

percentual_desconto_usado: (Comportamental) É um "caçador de promoções"?

In [ ]:
print("ETAPA 1: Criando dataset fictício...")

num_clientes = 300

data = {
    'id_cliente': range(1, num_clientes + 1),
    'dias_desde_ultima_compra': np.random.randint(1, 365, num_clientes),
    'gasto_total_ano': np.random.normal(loc=1000, scale=400, size=num_clientes).clip(50), # média 1000
    'frequencia_compras_ano': np.random.randint(1, 50, num_clientes),
    'percentual_desconto_usado': np.random.rand(num_clientes).clip(0, 0.5) # De 0% a 50%
}

df_clientes = pd.DataFrame(data)

# Para tornar os clusters mais interessantes, vamos "forçar" alguns perfis:
# Cluster 0 (VIPs): 50 clientes que gastam muito, compram sempre, desconto baixo
df_clientes.loc[0:50, 'gasto_total_ano'] = np.random.normal(loc=3500, scale=500, size=51)
df_clientes.loc[0:50, 'frequencia_compras_ano'] = np.random.randint(40, 70, 51)
df_clientes.loc[0:50, 'dias_desde_ultima_compra'] = np.random.randint(1, 30, 51)

# Cluster 1 (Em Risco): 50 clientes que gastam pouco, não compram há muito tempo
df_clientes.loc[51:100, 'gasto_total_ano'] = np.random.normal(loc=500, scale=100, size=50)
df_clientes.loc[51:100, 'frequencia_compras_ano'] = np.random.randint(1, 5, 50)
df_clientes.loc[51:100, 'dias_desde_ultima_compra'] = np.random.randint(200, 365, 50)

# O resto (101-299) são clientes "médios" ou "caçadores de promoção"
df_clientes.loc[101:200, 'percentual_desconto_usado'] = np.random.rand(100).clip(0.4, 0.8) # Usam muito desconto

print("Dataset criado com sucesso:")
print(df_clientes.describe())

In [ ]:
print("ETAPA 1: Criando dataset fictício...")

num_clientes = 300

data = {
    'id_cliente': range(1, num_clientes + 1),
    'dias_desde_ultima_compra': np.random.randint(1, 365, num_clientes),
    'gasto_total_ano': np.random.normal(loc=1000, scale=400, size=num_clientes).clip(50), # média 1000
    'frequencia_compras_ano': np.random.randint(1, 50, num_clientes),
    'percentual_desconto_usado': np.random.rand(num_clientes).clip(0, 0.5) # De 0% a 50%
}

df_clientes = pd.DataFrame(data)

# Para tornar os clusters mais interessantes, vamos "forçar" alguns perfis:
# Cluster 0 (VIPs): 50 clientes que gastam muito, compram sempre, desconto baixo
df_clientes.loc[0:50, 'gasto_total_ano'] = np.random.normal(loc=3500, scale=500, size=51)
df_clientes.loc[0:50, 'frequencia_compras_ano'] = np.random.randint(40, 70, 51)
df_clientes.loc[0:50, 'dias_desde_ultima_compra'] = np.random.randint(1, 30, 51)

# Cluster 1 (Em Risco): 50 clientes que gastam pouco, não compram há muito tempo
df_clientes.loc[51:100, 'gasto_total_ano'] = np.random.normal(loc=500, scale=100, size=50)
df_clientes.loc[51:100, 'frequencia_compras_ano'] = np.random.randint(1, 5, 50)
df_clientes.loc[51:100, 'dias_desde_ultima_compra'] = np.random.randint(200, 365, 50)

# O resto (101-299) são clientes "médios" ou "caçadores de promoção"
df_clientes.loc[101:200, 'percentual_desconto_usado'] = np.random.rand(100).clip(0.4, 0.8) # Usam muito desconto

print("Dataset criado com sucesso:")
print(df_clientes.describe())

Etapa 2: Pré-Processamento (StandardScaler)

Esta é a etapa mais importante do clustering.

O K-Means funciona medindo a "distância" entre os pontos. A feature gasto_total_ano (valores de 50 a 3500) teria um "peso" mil vezes maior que percentual_desconto_usado (valores de 0 a 1). Isso distorceria totalmente nosso modelo.

O StandardScaler resolve isso. Ele transforma todas as nossas features para que elas tenham a mesma escala (média 0 e desvio padrão 1), garantindo que todas tenham a mesma importância inicial para o modelo.

In [ ]:
print("ETAPA 2: Padronizando a escala dos dados...")

# Selecionamos apenas as colunas que descrevem o comportamento
features = ['dias_desde_ultima_compra', 'gasto_total_ano', 'frequencia_compras_ano', 'percentual_desconto_usado']
df_features = df_clientes[features]

# 1. Instanciar o Scaler
scaler = StandardScaler()

# 2. Treinar e Transformar os dados
# O .fit() aprende a média e o desvio
# O .transform() aplica a padronização
df_scaled = scaler.fit_transform(df_features)

print("Dados padronizados (média 0, desvio 1):")
print(df_scaled[:5]) # Mostra as 5 primeiras linhas

Etapa 3: Encontrando o Número Ideal de Clusters (O Método do Cotovelo)

Nós não sabemos quantos "perfis" de clientes existem. São 2? 3? 5?

O "Método do Cotovelo" (Elbow Method) nos ajuda a decidir. Vamos rodar o K-Means com vários 'K's (de 1 a 10) e calcular a "Inércia" (a soma total das distâncias dos clientes até o centro do seu cluster).

Procuramos o ponto no gráfico onde a linha "quebra" (o 'cotovelo'). Este é o ponto onde adicionar mais um cluster não traz muito mais benefício (a inércia não cai tanto).

In [ ]:
print("ETAPA 3: Método do Cotovelo (Elbow Method)...")

inertia = []
K_range = range(1, 11)

for k in K_range:
    # n_init=10 evita resultados ruins por sorteio inicial
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    model.fit(df_scaled)
    inertia.append(model.inertia_)

# Plotar o gráfico do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertia, marker='o', linestyle='--')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xticks(K_range)
plt.grid(True)
plt.show()

Etapa 4: Treinamento do Modelo K-Means

O gráfico do cotovelo (como forçamos os dados) deve mostrar uma "quebra" clara em K=3 ou K=4. Para este exercício, vamos assumir que k=4 é o número ideal de perfis de clientes.

Agora vamos treinar o modelo K-Means final com k=4 e usar o .labels_ para "carimbar" cada cliente com o seu rótulo de cluster (0, 1, 2, ou 3).

In [ ]:
print("ETAPA 4: Treinando o modelo K-Means final...")

k_ideal = 4

# 1. Instanciar o modelo
model_kmeans = KMeans(n_clusters=k_ideal, random_state=42, n_init=10)

# 2. Treinar o modelo com os dados padronizados
model_kmeans.fit(df_scaled)

# 3. Pegar os rótulos (os grupos) e salvar no DataFrame original
df_clientes['cluster'] = model_kmeans.labels_

print("DataFrame final com os clusters atribuídos:")
print(df_clientes.head())

# Verificar quantos clientes em cada cluster
print("\nContagem de clientes por cluster:")
print(df_clientes['cluster'].value_counts())

Etapa 5: Análise dos Clusters (A Psicologia / O Insight)

O modelo técnico está pronto. Agora vem o trabalho do Cientista de Dados Comportamental: Interpretar o que esses clusters significam.

Vamos calcular a média de cada feature (gasto_total, frequencia, etc.) para cada cluster e entender o "perfil" de cada grupo.

Para visualizar os grupos (já que não podemos plotar 4 dimensões), usaremos PCA (Análise de Componentes Principais) para "comprimir" nossas 4 features em 2 eixos (PC1 e PC2) e plotar em um gráfico 2D.

In [ ]:
print("ETAPA 5: Interpretando os perfis dos clusters...")

# 1. Análise de Perfil (A Média de cada cluster)
# Nós usamos 'df_features' (o original, não padronizado) para interpretar
# os resultados em valores reais (ex: R$ 3500)
cluster_profile = df_clientes.groupby('cluster')[features].mean()

print("--- Perfil Médio de Cada Cluster ---")
print(cluster_profile)

# 2. Visualização 2D com PCA
print("\nGerando visualização 2D dos clusters...")

# Instanciar o PCA
pca = PCA(n_components=2)

# Aplicar o PCA nos dados padronizados
df_pca = pca.fit_transform(df_scaled)

# Salvar os componentes no DataFrame
df_clientes['pc1'] = df_pca[:, 0]
df_clientes['pc2'] = df_pca[:, 1]

# Plotar o scatterplot
plt.figure(figsize=(12, 8))
sns.scatterplot(x='pc1', y='pc2', hue='cluster', data=df_clientes, palette='deep', s=100, alpha=0.8)
plt.title('Visualização 2D dos Clusters de Clientes (via PCA)')
plt.xlabel('Componente Principal 1 (PC1)')
plt.ylabel('Componente Principal 2 (PC2)')
plt.legend(title='Cluster')
plt.show()

Etapa 6: Conclusão e Personas (A Entrega de Negócio)

Com base na tabela cluster_profile (as médias de cada grupo), traduzimos os números em Personas acionáveis para a equipe de Marketing da @mepresa.

(Nota: Os perfis abaixo são uma interpretação dos dados fictícios que criamos. Os seus podem variar um pouco.)

Cluster 0 (Ex: Gasto Alto, Frequência Alta, Recência Baixa):

Persona: "Clientes VIP"

Comportamento: Nossos melhores clientes. Gastam muito, compram sempre e compraram recentemente.

Ação de Negócio (Marketing): Programas de fidelidade, acesso antecipado a lançamentos, brindes de luxo. Não enviar descontos (eles não precisam).

Cluster 1 (Ex: Gasto Baixo, Frequência Baixa, Recência Alta):

Persona: "Clientes em Risco (Churn)"

Comportamento: Eram clientes, mas gastaram pouco e não compram há muito tempo (Recência alta).

Ação de Negócio (Retenção): Campanha de reengajamento "Sentimos sua falta!" com um cupom de desconto agressivo para reativá-los.

Cluster 2 (Ex: Gasto Médio, Frequência Média, Alto Desconto):

Persona: "Caçadores de Promoção"

Comportamento: Clientes que só compram quando há promoções. O percentual_desconto_usado deles é o mais alto.

Ação de Negócio (Financeiro): Excluir este grupo das campanhas de desconto mais caras. Enviar apenas promoções de "leve 3 pague 2" (foco em volume, não em margem).

Cluster 3 (Ex: Gasto Médio, Frequência Baixa, Recência Média):

Persona: "Clientes Regulares / Oportunidade"

Comportamento: O cliente "médio". Compra de vez em quando, gasta um valor razoável.

Ação de Negócio (Upsell): Campanha de cross-sell e upsell. Sugerir produtos de novas categorias (Ex: "Vimos que você adora maquiagem, que tal conhecer nossa linha de pele?").